In [1]:
import re
import os
import math
import pickle
import datetime
import matplotlib
import numpy as np
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
from reportlab.lib import colors
from sklearn import preprocessing
from reportlab.lib.units import mm, inch
from reportlab.lib.enums import TA_CENTER
from sklearn.linear_model import LinearRegression
from reportlab.lib.pagesizes import elevenSeventeen
import ipywidgets as widgets	# 控件库
from IPython.display import display	# 显示控件的方法
from reportlab.platypus import SimpleDocTemplate, Spacer, Image
from scipy import stats
import logging as logger
import matplotlib.pyplot as plt
from scipy.sparse import csgraph
from sklearn import preprocessing
from scipy.sparse import csr_matrix
import scipy.spatial.distance as dis
from pandas.core.common import flatten
matplotlib.use('Agg')
logger = logger.Logger("debug")

In [2]:
HOME_FOLDER = '/Users/hiCore/Develop/Workspace_Jupyter/Expedia/Simplification'
os.chdir(HOME_FOLDER)
Observe = 'CostAmt'
LENGTH_OF_STAY_DAY_CNT = 1
HOTEL_PATTERN_LOS = 1
HOTEL_PATTERN_PERSONCNT = 2
HOTEL_PATTERN_DATAVERSION = 2
HOTEL_PATTERN_RATEPLANLEVEL = 0
HOTEL_PATTERN_INPUT_FOLDER2 = os.getcwd()+'/Data{}/'.format(HOTEL_PATTERN_DATAVERSION)
HOTEL_PATTERN_INPUT_FOLDER = os.getcwd()+'/Data/'
HOTEL_PATTERN_OUTPUT_FOLDER = os.getcwd()+'/Result/MINE2/'
HOTEL_PATTERN_Observes = ['CostAmt', 'PriceAmt', 'LARAmt', 'LARMarginAmt', 'LARTaxesAndFeesAmt']
PATTERN_ATTRIBUTE_CLEANUP_OUTPUT = True
PATTERN_ATTRIBUTE_OUTPUT_FOLDER = os.getcwd()+'/Result/DAG.nosync/'
PATTERN_ATTRIBUTE_INPUT_FOLDER = os.getcwd()+'/Data/'
PATTERN_ATTRIBUTE_INPUT_FOLDER2 = os.getcwd() + '/Result/MINE2/'
PATTERN_MAPPING_INPUT_FOLDER = os.getcwd()+'/Data2/'
PATTERN_MAPPING_OUTPUT_FOLDER = os.getcwd()+'/Result/MINE2/'
PATTERN_MAPPING_INPUT_FOLDER2 = os.getcwd()+'/Result/MINE2/'
COL_CFG = pd.DataFrame(
    np.array(
        [['RatePlanID', 'None', 1],
         ['RatePlanTypeID', 'Dice', 1],
         ['RoomTypeID', 'None', 1],
         ['ActiveStatusTypeID', 'None', 1],
         ['RatePlanCodeSupplier', 'Dice', 1],
         ['ManageOnExtranetBool', 'cityblock', 1],
         ['CostCodeDefault', 'Dice', 1],
         ['AllowInventoryLimitEditBool', 'cityblock', 1],
         ['RatePlanIDOriginal', 'None', 1],
         ['ARIEnabledBool', 'cityblock', 1],
         ['WaiveTaxesBool', 'cityblock', 1],
         ['SKUGroupFeeSetID', 'Dice', 1],
         ['SKUGroupCancelPolicySetID', 'Dice', 1],
         ['SuppressionOverrideBool', 'cityblock', 1],
         ['RatePlanIDOriginalDC', 'None', 1],
         ['SKUGroupMarginRuleSetID', 'Dice', 1],
         ['ARIRolloutBool', 'cityblock', 1],
         ['RatePlanCostPriceTypeID', 'Dice', 1],
         ['DOACostPriceBool', 'cityblock', 1],
         ['LOSCostPriceBool', 'cityblock', 1],
         ['SpecialDiscountPercent', 'cityblock', 1],
         ['SuppressionOverrideBool', 'None', 1],
         ['BusinessModelMask', 'Dice', 1],
         ['CostCodeDefaultAgency', 'Dice', 1],
         ['SKUGroupMarginRuleSetIDAgency', 'Dice', 1],
         ['DepositRequiredBool', 'cityblock', 1],
         ['SyncBookingOverrideBool', 'cityblock', 1],
         ]),
    columns=['name', 'algo', 'weight'])
OUTPUT_RESULT_FILE_NAME = "{}_mapping_result.pdf"
OUTPUT_LINEAR_FILE_NAME = "{}{}_linear_result.jpg"

In [3]:
### Search Level
global global_enable_cache
global_enable_cache = "Disable"
drpbx1 = widgets.Dropdown(options=[("Enable","Enable"),("Disable","Disable")], index=0, value="Disable", label="Disable")
def chosen1(_):
    print("Selected index:{}, value:{}, label:{}".format(drpbx1.index, drpbx1.value, drpbx1.label))
    global global_enable_cache
    global_enable_cache = drpbx1.value
drpbx1.observe(chosen1, names="value")
print("EnableCache:")
display(drpbx1)

EnableCache:


Dropdown(index=1, options=(('Enable', 'Enable'), ('Disable', 'Disable')), value='Disable')

In [4]:
### Search Level
global global_search_level
global_search_level = "Room"
drpbx2 = widgets.Dropdown(options=[("Room","Room"),("Hotel","Hotel")], index=0, value="Room", label="Room")
def chosen2(_):
    print("Selected index:{}, value:{}, label:{}".format(drpbx2.index, drpbx2.value, drpbx2.label))
    global global_search_level
    global_search_level = drpbx2.value
drpbx2.observe(chosen2, names="value")
print("Search Level:")
display(drpbx2)

Search Level:


Dropdown(options=(('Room', 'Room'), ('Hotel', 'Hotel')), value='Room')

In [5]:
# search_level = "Room"
# enableCache = False
# search_id = 166628
# search_level = "Hotel"
# enableCache = False
# search_id = 16639
global global_search_id
if global_search_level == "Room":
    ### ROOM TPYE
    global_search_id = 166628
    drpbx3 = widgets.Dropdown(options=[("166629	(14)",166629),
    ("166628	(13)",166628),
    ("193407	(13)",193407),
    ("193410	(13)",193410),
    ("193413	(13)",193413),
    ("271897	(13)",271897),
    ("201119434	(13)",201119434),
    ("201119443	(13)",201119443),
    ("200160226	(12)",200160226),
    ("200160302	(12)",200160302),
    ("200160344	(12)",200160344),
    ("200160364	(12)",200160364),
    ("201119429	(12)",201119429),
    ("201119439	(12)",201119439),
    ("200160278	(11)",200160278),
    ("200160421	(11)",200160421)], index=0, value=166628, label="166628	(13)")
    def chosen3(_):
        print("Selected index:{}, value:{}, label:{}".format(drpbx3.index, drpbx3.value, drpbx3.label))
        global global_search_id
        global_search_id = drpbx3.value
    drpbx3.observe(chosen3, names="value")
    print("Room Id:")
    display(drpbx3)
else:
    ## Hotel ID
    global_search_id = 16639
    drpbx = widgets.Dropdown(options=[("16639",16639),
    ("862",862),
    ("1797",1797),
    ("6362",6362),
    ("12079",12079),
    ("12160",12160),
    ("12800",12800),
    ("14388",14388),
    ("15144",15144),
    ("15212",15212),
    ("19692",19692),
    ("19961",19961),
    ("21327",21327),
    ("22673",22673),
    ("24662",24662),
    ("27740",27740),
    ("42448",42448),
    ("42526",42526),
    ("50886",50886),
    ("60561",60561),
    ("67394",67394),
    ("67970",67970),
    ("197433",197433),
    ("208594",208594),
    ("215262",215262),
    ("281521",281521),
    ("281536",281536),
    ("281920",281920),
    ("281939",281939),
    ("297146",297146),
    ("328177",328177),
    ("424868",424868),
    ("426205",426205),
    ("454849",454849),
    ("519720",519720),
    ("519726",519726),
    ("531180",531180),
    ("531190",531190),
    ("538638",538638),
    ("787571",787571),
    ("890490",890490),
    ("909717",909717),
    ("973778",973778),
    ("1074382",1074382),
    ("1155964",1155964),
    ("1172342",1172342),
    ("1191979",1191979),
    ("1228332",1228332),
    ("1246232",1246232),
    ("1321942",1321942),
    ("1545120",1545120),
    ("1601018",1601018),
    ("1636325",1636325),
    ("1781224",1781224),
    ("1784929",1784929),
    ("1793627",1793627),
    ("1844848",1844848),
    ("2009510",2009510),
    ("2009515",2009515),
    ("2046329",2046329),
    ("2058961",2058961),
    ("2147324",2147324),
    ("2150460",2150460),
    ("2163016",2163016),
    ("2191649",2191649),
    ("2239837",2239837),
    ("2246517",2246517),
    ("2270665",2270665),
    ("2270677",2270677),
    ("2292826",2292826),
    ("2351486",2351486),
    ("2406330",2406330),
    ("2418923",2418923),
    ("2419417",2419417),
    ("2597717",2597717),
    ("2602846",2602846),
    ("2774577",2774577),
    ("2918077",2918077),
    ("3508863",3508863),
    ("3859038",3859038),
    ("5977476",5977476),
    ("6257605",6257605),
    ("6282047",6282047),
    ("6384898",6384898),
    ("6502660",6502660),
    ("6828402",6828402),
    ("7362466",7362466),
    ("7714604",7714604),
    ("7757367",7757367),
    ("8150608",8150608),
    ("8362336",8362336),
    ("8474909",8474909),
    ("8731496",8731496),
    ("8745457",8745457),
    ("9261405",9261405),
    ("18109739",18109739),
    ("23251275",23251275),
    ("23379342",23379342),
    ("23830678",23830678),
    ("23977136",23977136),
    ("27041847",27041847),
    ("27041849",27041849),
    ("27238090",27238090),
    ("27238095",27238095),
    ("27373038",27373038),
    ("29098521",29098521),
    ("30073675",30073675),
    ("30435488",30435488),
    ("30449392",30449392),
    ("30473613",30473613),
    ("31356803",31356803),
    ("32749220",32749220),
    ("32911452",32911452),
    ("33213151",33213151),
    ("35129611",35129611),
    ("35521694",35521694),
    ("35562140",35562140),
    ("36447066",36447066),
    ("36501393",36501393),
    ("38318741",38318741),
    ("41979240",41979240),
    ("42839826",42839826),
    ("45970429",45970429),
    ("45976006",45976006),
    ("48251410",48251410),
    ("50706495",50706495),
    ("51263687",51263687),
    ("53635112",53635112),
    ("54845071",54845071),
    ("55259840",55259840),
    ("55573268",55573268)], index=0, value=16639, label="16639")
    def chosen(_):
        print("Selected index:{}, value:{}, label:{}".format(drpbx.index, drpbx.value, drpbx.label))
        global global_search_id
        global_search_id = drpbx.value
    drpbx.observe(chosen, names="value")
    print("Hotel Id:")
    display(drpbx)

Hotel Id:


Dropdown(options=(('16639', 16639), ('862', 862), ('1797', 1797), ('6362', 6362), ('12079', 12079), ('12160', …

In [6]:
search_id = global_search_id
search_level = global_search_level
enableCache = global_enable_cache
print("search_id: {}".format(search_id))
print("search_level: {}".format(search_level))
print("enableCache: {}".format(enableCache))

search_id: 16639
search_level: Hotel
enableCache: Enable


In [7]:
def read_file_dbo_RoomType_NoIdent_by_room_id(room_id):
    logger.debug("read file dbo_RoomType_NoIdent.csv")
    read_data_rt = pd.read_csv(HOTEL_PATTERN_INPUT_FOLDER + 'dbo_RoomType_NoIdent.csv', encoding='utf-8', sep=',',
                               engine='python',
                               header=0).fillna(0)
    read_data_rt = read_data_rt[['SKUGroupID', 'RoomTypeID', 'ActiveStatusTypeID']]
    read_data_rt = read_data_rt.loc[read_data_rt['ActiveStatusTypeID'] == 2]
    read_data_rt.drop(['ActiveStatusTypeID'], axis=1, inplace=True)
    logger.debug(read_data_rt.head(10))
    read_data_rt = read_data_rt.loc[read_data_rt['RoomTypeID'].isin([room_id])]
    return read_data_rt

In [8]:
def read_file_dbo_RoomType_NoIdent(hotel_id):
    logger.debug("read file dbo_RoomType_NoIdent.csv")
    read_data_rt = pd.read_csv(HOTEL_PATTERN_INPUT_FOLDER + 'dbo_RoomType_NoIdent.csv', encoding='utf-8', sep=',',
                               engine='python',
                               header=0).fillna(0)
    read_data_rt = read_data_rt[['SKUGroupID', 'RoomTypeID', 'ActiveStatusTypeID']]
    read_data_rt = read_data_rt.loc[read_data_rt['ActiveStatusTypeID'] == 2]
    read_data_rt.drop(['ActiveStatusTypeID'], axis=1, inplace=True)
    logger.debug(read_data_rt.head(10))
    read_data_rt = read_data_rt.loc[read_data_rt['SKUGroupID'].isin([hotel_id])]
    return read_data_rt

In [9]:
def read_file_dboRatePlanNoIdent(room_type_id_list):
    logger.debug("read file dbo_RatePlan_NoIdent.csv")
    read_data_rp = pd.read_csv(HOTEL_PATTERN_INPUT_FOLDER + 'dbo_RatePlan_NoIdent.csv', encoding='utf-8', sep=',',
                               engine='python',
                               header=0).fillna(0)
    read_data_rp.drop(['UpdateTPID', 'ChangeRequestID', 'UpdateTUID'], axis=1, inplace=True)
    read_data_rp.drop(['UpdateDate', 'LastUpdatedBy', 'UpdateClientID', 'RatePlanLogID'], axis=1, inplace=True)
    read_data_rp = read_data_rp.loc[(read_data_rp['ActiveStatusTypeID'] == 2) & (read_data_rp['RoomTypeID'].isin(room_type_id_list))]
    return read_data_rp

In [10]:
def read_file_RatePlanLevelCostPrice(hotel_id, read_data_rp_id_list):
    '''
     #     RatePlanID,StayDate,RatePlanLevel,PersonCnt,LengthOfStayDayCnt,ActiveStatusTypeID,
     #     RatePlanLevelCostPriceLogSeqNbr,CostAmt,PriceAmt,CostCode,ChangeRequestIDOld,
     #     SupplierUpdateDate,SupplierUpdateTPID,SupplierUpdateTUID,UpdateDate,SupplierLogSeqNbr,
     #     ChangeRequestID,LARAmt,LARMarginAmt,LARTaxesAndFeesAmt
    :param hotel_id:
    :param read_data_rp:
    :return:
    '''
    logger.debug("read file {}_RatePlanLevelCostPrice.csv.zip".format(hotel_id))
    read_data = pd.read_csv(HOTEL_PATTERN_INPUT_FOLDER2 + str(hotel_id) + '_RatePlanLevelCostPrice.csv.zip',
                            sep=',', engine='python',
                            header=0).fillna(0)
    read_data = read_data.loc[read_data['RatePlanID'].isin(read_data_rp_id_list)]
    logger.debug(read_data)
    read_data.drop(['ActiveStatusTypeID', 'RatePlanLevelCostPriceLogSeqNbr', 'ChangeRequestIDOld'], axis=1,
                   inplace=True)
    read_data.drop(['SupplierUpdateDate', 'SupplierUpdateTPID', 'SupplierUpdateTUID'], axis=1, inplace=True)
    read_data.drop(['UpdateDate', 'SupplierLogSeqNbr', 'ChangeRequestID'], axis=1, inplace=True)
    read_data = read_data.loc[(read_data['RatePlanLevel'] == HOTEL_PATTERN_RATEPLANLEVEL) & (
                read_data['LengthOfStayDayCnt'] == HOTEL_PATTERN_LOS)
                              & (read_data['PersonCnt'] == HOTEL_PATTERN_PERSONCNT)]
    read_data.drop(['RatePlanLevel', 'LengthOfStayDayCnt', 'PersonCnt'], axis=1, inplace=True)
    return read_data

In [11]:
def read_rt_rp_by_room_id(room_id):
    read_data_rt = read_file_dbo_RoomType_NoIdent_by_room_id(room_id)
    read_data_rp = read_file_dboRatePlanNoIdent([room_id])
    return read_data_rt, read_data_rp

In [12]:
def read_rt_rp_by_hotel_id(hotel_id):
    read_data_rt = read_file_dbo_RoomType_NoIdent(hotel_id)
    read_data_rp = read_file_dboRatePlanNoIdent(read_data_rt['RoomTypeID'].values.tolist())
    return read_data_rt, read_data_rp

In [13]:
def read_csv_data_and_filter_by_room_type_id(room_type_id, hotel_id):
    '''读取CSV文件并进行过滤
        First step: 根据 HotelID 获取所有 RoomTypeId (read_data_rt)
            读取./Data/dbo_RoomType_NoIdent.csv文件 = read_data_rt
            取出 'SKUGroupID', 'RoomTypeID', 'ActiveStatusTypeID' 列
            选择 ActiveStatusTypeID == 2 并删除ActiveStatusTypeID列
            选择 SKUGroupID 为指定 HotelID 的数据 = read_data_rt

        Second step: 根据 RoomTypeId 获取对应的 RatePlanID (read_data_rp)
            读取./Data/dbo_RatePlan_NoIdent.csv 数据文件 = read_data_rp
            过滤 ActiveStatusTypeID == 2 并且 RoomTypeID == read_data_rt 的 RoomTypeID 的数据并取出它的 RatePlanID  = read_data_rp

        Third step: 根据RatePlanID 获取 CostPrice (read_data)
            从 ./Data2/HotelID_RatePlanLevelCostPrice.csv.zip 压缩文件中读取数据
            选出 RatePlanID 在 read_data_rp['RatePlanID'] 中的数据
            删除 'ActiveStatusTypeID', 'RatePlanLevelCostPriceLogSeqNbr', 'ChangeRequestIDOld', 'SupplierUpdateDate', 'SupplierUpdateTPID', 'SupplierUpdateTUID', 'UpdateDate', 'SupplierLogSeqNbr', 'ChangeRequestID' 列
            选出 'RatePlanLevel' == HOTEL_PATTERN_RATEPLANLEVEL 'LengthOfStayDayCnt' == HOTEL_PATTERN_LOS 'PersonCnt' == HOTEL_PATTERN_PERSONCNT 的数据并删除这几列  = read_data
        Fourth step:
    '''
    # read_data_rt = read_file_dbo_RoomType_NoIdent_by_room_id(room_type_id)
    read_data_rp = read_file_dboRatePlanNoIdent([room_type_id])
    # hotel_id = read_data_rt['SKUGroupID'].values.tolist()[0]
    read_data_rp_id_list = read_data_rp['RatePlanID']
    read_data = read_file_RatePlanLevelCostPrice(hotel_id, read_data_rp_id_list)
    read_data = zscore_filter(read_data)
    return read_data_rp, read_data

In [14]:
def read_csv_data_and_filter(hotel_id):
    '''读取CSV文件并进行过滤
        First step: 根据 HotelID 获取所有 RoomTypeId (read_data_rt)
            读取./Data/dbo_RoomType_NoIdent.csv文件 = read_data_rt
            取出 'SKUGroupID', 'RoomTypeID', 'ActiveStatusTypeID' 列
            选择 ActiveStatusTypeID == 2 并删除ActiveStatusTypeID列
            选择 SKUGroupID 为指定 HotelID 的数据 = read_data_rt

        Second step: 根据 RoomTypeId 获取对应的 RatePlanID (read_data_rp)
            读取./Data/dbo_RatePlan_NoIdent.csv 数据文件 = read_data_rp
            过滤 ActiveStatusTypeID == 2 并且 RoomTypeID == read_data_rt 的 RoomTypeID 的数据并取出它的 RatePlanID  = read_data_rp

        Third step: 根据RatePlanID 获取 CostPrice (read_data)
            从 ./Data2/HotelID_RatePlanLevelCostPrice.csv.zip 压缩文件中读取数据
            选出 RatePlanID 在 read_data_rp['RatePlanID'] 中的数据
            删除 'ActiveStatusTypeID', 'RatePlanLevelCostPriceLogSeqNbr', 'ChangeRequestIDOld', 'SupplierUpdateDate', 'SupplierUpdateTPID', 'SupplierUpdateTUID', 'UpdateDate', 'SupplierLogSeqNbr', 'ChangeRequestID' 列
            选出 'RatePlanLevel' == HOTEL_PATTERN_RATEPLANLEVEL 'LengthOfStayDayCnt' == HOTEL_PATTERN_LOS 'PersonCnt' == HOTEL_PATTERN_PERSONCNT 的数据并删除这几列  = read_data
        Fourth step:
    '''
    read_data_rt = read_file_dbo_RoomType_NoIdent(hotel_id)
    read_data_rp = read_file_dboRatePlanNoIdent(read_data_rt['RoomTypeID'].values.tolist())
    read_data_rp_id_list = read_data_rp['RatePlanID']
    read_data = read_file_RatePlanLevelCostPrice(hotel_id, read_data_rp_id_list)
    read_data = zscore_filter(read_data)
    return read_data_rt, read_data_rp, read_data

In [15]:
def zscore_filter(read_data):
    read_data['z_score'] = stats.zscore(read_data[Observe])
    logger.debug(read_data.head(20))
    read_data = read_data.loc[read_data['z_score'].abs() <= 3]
    return read_data

In [16]:
def get_connected_components(read_data):
    read_data_gp = read_data[['StayDate', Observe, 'RatePlanID']].groupby(['RatePlanID'], sort=False)
    df_corr = pd.DataFrame()
    for name, group in read_data_gp:
        group.reset_index(drop=True, inplace=True)
        df_corr[name] = group.set_index('StayDate')[Observe]
    # https://blog.csdn.net/walking_visitor/article/details/85128461
    # 默认使用 pearson 相关系数计算方法，但这种方式存在误判
    # df_corr.fillna(0, inplace=True)
    # 删除缺失值比例大于30%
    s1 = (df_corr.isnull().sum() / df_corr.shape[0]) >= 0.3  # 得到缺失值的比例大于30%
    df_corr = df_corr[s1[s1 == False].index.tolist()]  # 删除比例大于30%的缺失值

    for column in list(df_corr.columns[df_corr.isnull().sum() > 0]):
        mean_val = df_corr[column].mean()
        df_corr[column].fillna(mean_val, inplace=True)
    df_corr = df_corr.corr()
    np.fill_diagonal(df_corr.values, 0)
    graph = csr_matrix(df_corr >= 0.99)
    n, labels = csgraph.connected_components(graph)
    logger.debug('The number of connected components: {} || The number of groupby[RatePlanID]:{}'.format(n,read_data_gp.ngroups))
    return n, labels, df_corr

In [28]:
def generate_group_file_and_img(read_data, as_file_name_search_id):
    '''生成结果到CSV文件
        First Step: 对观测值的RatePlanID进行分组 (read_data_gp)
            创建一个空的 DataFrame 来存储数据 = df_cdist
            分别从read_data(CostPrice文件)对 'CostAmt', 'PriceAmt', 'LARAmt', 'LARMarginAmt', 'LARTaxesAndFeesAmt' 计算 zscore 分数并存储到 read_data 文件的 zscore 列
            过滤掉 read_data z_score 分数小于等于 3 的数据
            然后根据 RatePlanID 进行分组，分组后取出 'StayDate', Observe, 'RatePlanID' 这三列 = read_data_gp
            遍历分组后的数据，并对分组数据的索引重置为日期(StayDate) = df_corr
            对 df_corr 计算相关系数,矩阵对角填充为0 并选出相关系数大于95% 的图
        Second Step: 使用稀疏矩阵图对不同的观测值进行 RatePlanId 分组并保存结果到文件 (df_cdist)
            分析稀疏图的连通分量 获取 连通域个数 及 连接组件的标签长度
            log 打印 连通图分组个数 及 根据RatePlanID 的分组个数
            遍历分组结果,选取 read_data 的 RatePlanID 在稀疏矩阵图中的分组结果
            对 read_data 的 'StayDate', 'RatePlanID' 进行分组, 并对 Observe 的金额进行求和并画图
            将 df_cdist 分组结果 [观测值，分组id,分组的RatePlanId数组]  输出到 ./Result/MINE2/HotelID_patterngroup.csv 文件中
    '''

    # for Observe in HOTEL_PATTERN_Observes:
    #     if Observe != "CostAmt":
    #         continue
    n, labels, df_corr = get_connected_components(read_data)
    df_cdist = pd.DataFrame()

    if n <= 0:
        return df_cdist, 0

    count = 0
    row_size = math.ceil(n ** 0.5)
    column_size = math.ceil(n / row_size)
    fig, axes = plt.subplots(row_size, column_size, figsize=(column_size*4, row_size*5))
    plt.subplots_adjust(left=0.125, bottom=0.04, right=0.9, top=1, hspace=0.1, wspace=0.2)
    for i in range(row_size):
        for j in range(0, column_size):
            if count >= n:
                continue
            nodes = df_corr.index[np.where(labels == count)]
            df_cdist = df_cdist.append([[Observe, count, nodes.values]], ignore_index=True)
            if column_size == 1 and row_size == 1:
                read_data.loc[(read_data['RatePlanID'].isin(nodes))].groupby(['StayDate', 'RatePlanID']).sum()[
                    Observe].unstack().plot(ax=axes)
            if column_size == 1 and row_size > 1:
                read_data.loc[(read_data['RatePlanID'].isin(nodes))].groupby(['StayDate', 'RatePlanID']).sum()[
                    Observe].unstack().plot(ax=axes[i])
            if column_size > 1 and row_size > 1:
                read_data.loc[(read_data['RatePlanID'].isin(nodes))].groupby(['StayDate', 'RatePlanID']).sum()[
                    Observe].unstack().plot(ax=axes[i][j])
            # read_data.loc[(read_data['RatePlanID'].isin(nodes))].groupby(['StayDate', 'RatePlanID']).sum()[
            #     Observe].unstack().plot(ax=axes[i][j])
            logger.debug("left compatue length: {}".format(n - count))
            count += 1
    plt.savefig('{}{}_all_pattern_group.png'.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, as_file_name_search_id), dpi=200, bbox_inches='tight')

    df_cdist.columns = ['Observe', 'GroupID', 'Group']
    df_cdist_copy = df_cdist.copy()
    df_cdist_copy["Group"] = df_cdist_copy.apply(lambda x: x["Group"].tolist(), axis=1)
    df_cdist_copy["RatePlanLen"] = df_cdist_copy.apply(lambda x: len(x["Group"]), axis=1)
    best_group_id = np.random.choice(df_cdist_copy["RatePlanLen"][df_cdist_copy["RatePlanLen"] == df_cdist_copy["RatePlanLen"].max()].index)
    logger.debug("The best group is group_{}".format(int(best_group_id)+1))
    logger.debug("Generate {}'s grouping files".format(as_file_name_search_id))
    df_cdist_copy.to_csv('{}{}_patterngroup.csv'.format(HOTEL_PATTERN_OUTPUT_FOLDER, as_file_name_search_id), index=False)
    logger.debug("The generation of grouping files is complete")
    nodes = df_corr.index[np.where(labels == best_group_id)]

    # save sigle pattern group img
    fig, ax = plt.subplots(figsize=(18, 7))
    read_data.loc[(read_data['RatePlanID'].isin(nodes))].groupby(['StayDate', 'RatePlanID']).sum()[
        Observe].unstack().plot(ax=ax)
    # plt.tight_layout()
    plt.savefig('{}{}_patterngroup.png'.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, as_file_name_search_id))
    plt.show()
    plt.close()
    return df_cdist, best_group_id

In [18]:
if search_level == "Room":
    read_data_rt = read_file_dbo_RoomType_NoIdent_by_room_id(search_id)
    hotel_id = read_data_rt['SKUGroupID'].values.tolist()[0]
    print(hotel_id)
else:
    hotel_id = search_id

In [19]:
gpfile = '{}{}_patterngroup.csv'.format(HOTEL_PATTERN_OUTPUT_FOLDER, search_id)
all_pattern_group_image = '{}{}_all_pattern_group.png'.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, search_id)
gfimg = '{}{}_patterngroup.png'.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, search_id)
if os.path.exists(gpfile) and os.path.exists(gfimg) and os.path.exists(all_pattern_group_image) and enableCache=='true':
    logger.debug("Grouping file already exist!!")
    gp_df = pd.read_csv(gpfile, encoding='utf-8', sep=',', engine='python', header=0).fillna(0)
    best_group_id = np.random.choice(gp_df["RatePlanLen"][gp_df["RatePlanLen"] == gp_df["RatePlanLen"].max()].index)
    logger.debug("The best group index is group_{}".format(int(best_group_id)+1))
    global_bast_group_id = best_group_id
else:
    # Room级别: 在 dbo_RoomType_Noident.csv 文件中通过 RoomId 查询 HotelId
    if search_level == "Room":
        read_data_rp, read_data = read_csv_data_and_filter_by_room_type_id(search_id, hotel_id)
    if search_level == "Hotel":
        read_data_rt, read_data_rp, read_data = read_csv_data_and_filter(search_id)
    df_cdist, best_group_id = generate_group_file_and_img(read_data, search_id)
    global_bast_group_id = best_group_id

In [20]:
print(global_bast_group_id)

0


In [21]:
def generate_report(mappingFunctionResult):
    content = list()
    # 添加标题
    content.append(Graphs.draw_title())
    # 添加段落
    display_content = list()

    # 分组全图
    all_pattern_group_image = '{}{}_all_pattern_group.png'.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, search_id)
    if os.path.exists(all_pattern_group_image):
        content.append(Graphs.draw_text("Linear grouping diagram"))
        content.append(Graphs.draw_text(all_pattern_group_image))
        img = Image(all_pattern_group_image)
        img.drawHeight = (5*60) * mm
        img.drawWidth = 260 * mm
        img.hAlign = TA_CENTER
        content.append(img)

    # 分组图
    PatternGroupPngUrl = '{}{}_patterngroup.png'.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, search_id)
    if os.path.exists(PatternGroupPngUrl):
        content.append(Graphs.draw_text("The best group is group {}".format(int(global_bast_group_id)+1)))
        content.append(Graphs.draw_text('{}{}_patterngroup.csv'.format(HOTEL_PATTERN_OUTPUT_FOLDER, search_id)))
        content.append(Graphs.draw_text(PatternGroupPngUrl))
        # content.append(Spacer(1, 10 * mm))
        img = Image(PatternGroupPngUrl)
        img.drawHeight = 60 * mm
        img.drawWidth = 260 * mm
        img.hAlign = TA_CENTER
        content.append(img)

    # 线性方程图
    LinearPngUrl = OUTPUT_LINEAR_FILE_NAME.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, search_id)
    if os.path.exists(LinearPngUrl):
        content.append(Graphs.draw_text("Linear fitting equation diagram"))
        content.append(Graphs.draw_text(LinearPngUrl))
        # content.append(Spacer(1, 10 * mm))
        img = Image(LinearPngUrl)
        img.drawHeight = (6*50) * mm
        img.drawWidth = 260 * mm
        img.hAlign = TA_CENTER
        content.append(img)

    display_content.append("First we used the Mapping Function on Hotelid for {}.".format(search_id))
    display_content.append("In the personCnt={} case,".format(HOTEL_PATTERN_PERSONCNT))
    display_content.append(" lengthOfStayCnt = {}".format(LENGTH_OF_STAY_DAY_CNT))
    display_content.append(" and ratePlanLevel = {}".format(HOTEL_PATTERN_RATEPLANLEVEL))
    display_content.append(" and LOS = {}".format(HOTEL_PATTERN_LOS))
    display_content.append(" and Observe = {}".format(Observe))
    display_content.append(" and the best group is group {}".format((int(global_bast_group_id)+1)))

    content.append(Graphs.draw_text("RatePlan Attribute Relationship Graph"))
    pic_url = '{}{}_{}_pic.png'.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, 16639, 166628)
    if os.path.exists(pic_url):
        content.append(Spacer(1, 10 * mm))
        img = Image(pic_url)
        img.drawHeight = 60 * mm
        img.drawWidth = 200 * mm
        img.hAlign = TA_CENTER
        content.append(img)

    if mappingFunctionResult is not None and not mappingFunctionResult.empty:
        childTotalSizes = sum(mappingFunctionResult['ChildSize'])
        mappingFunctionTotalSize = sum(mappingFunctionResult['MappingFunctionSize'])
        compressionRatio = (childTotalSizes - mappingFunctionTotalSize) / childTotalSizes
        # 添加表格数据
        data = [('BaseRatePlan', 'childTotalSizes', "mappingFunctionTotalSize", 'Validation', 'CompressionRatio'),
                (mappingFunctionResult['BaseRP'][0], childTotalSizes, mappingFunctionTotalSize, all(mappingFunctionResult['Validation']), '{:.2%}'.format(compressionRatio))]
        display_content.append(" and Mapping Function compression ratio = {:.2%}.".format(compressionRatio))
        # 添加表
        content_text = "".join(display_content)
        content.append(Graphs.draw_text(content_text))
        content.append(Spacer(1, 10 * mm)) # 间隔区
        content.append(Graphs.draw_table(*data))
        # 添加bar装图
        max_value = max(childTotalSizes, mappingFunctionTotalSize)
        b_data = [[childTotalSizes, None], [None, mappingFunctionTotalSize]]
        ax_data = ['ChildSize', 'MappingFunctionSize']
        leg_items = [(colors.red, 'ChildTotalSize'), (colors.green, 'MappingFunctionTotalSize')]
        content.append(Spacer(1, 30 * mm))
        content.append(Graphs.draw_bar(b_data, ax_data, leg_items, max_value))

    # 生成pdf文件
    doc = SimpleDocTemplate(OUTPUT_RESULT_FILE_NAME.format(search_id), pagesize=elevenSeventeen, rightMargin=0.2 * inch,
                    leftMargin=0.2 * inch,
                    topMargin=10, bottomMargin=68)
    doc.build(content)

In [22]:
def read_group_file_method():
    read_data_group = pd.read_csv('{}{}_patterngroup.csv'.format(HOTEL_PATTERN_OUTPUT_FOLDER, search_id), encoding='utf-8', sep=',', engine='python', header=0).fillna(0)
    read_data_group = read_data_group.loc[(read_data_group['GroupID'] == global_bast_group_id) & (read_data_group['Observe'] == Observe)]
    return read_data_group

In [23]:
def read_and_preprocess_csv_file():
    '''
    读取 ./Result/MINE2/{hotel_id}_observe_gp.csv 文件
    :return:
    '''
    logger.debug("read_and_preprocess_group_csv_file begin")
    # read_data_rt = pd.read_csv('{}{}_{}_gp.csv'.format(PATTERN_ATTRIBUTE_INPUT_FOLDER2, self.hotel_id, self.observe), \
    #                    encoding='utf-8', sep=',', engine='python', header=0).fillna(0)
    #     RP2 = 260281795
    #     RP1 = 260282228
    read_group_file = read_group_file_method()
    read_data = pd.read_csv(PATTERN_MAPPING_INPUT_FOLDER + str(hotel_id) + '_RatePlanLevelCostPrice.csv.zip', sep=',', engine='python',
                            header=0).fillna(0)
    rate_plan_list_ids = np.array(literal_eval(read_group_file['Group'].iloc[0])).tolist()
    read_data = read_data.loc[read_data['RatePlanID'].isin(rate_plan_list_ids)]
    read_data = read_data.loc[(read_data['RatePlanLevel'] == HOTEL_PATTERN_RATEPLANLEVEL) & (read_data['LengthOfStayDayCnt'] == LENGTH_OF_STAY_DAY_CNT) \
                              & (read_data['PersonCnt'] == HOTEL_PATTERN_PERSONCNT)]
    # read_data = read_data[['StayDate', self.observe, 'RatePlanID']]
    read_data = read_data[['StayDate', 'LengthOfStayDayCnt', 'PersonCnt', Observe, 'RatePlanID']]
    logger.debug("read_and_preprocess_group_csv_file done")
    return rate_plan_list_ids, read_data

In [24]:
import datetime
class MappingFunction:
    def __init__(self):
        self.A = 0
        self.b = 0
        self.precision = 0
        self.exceptionPoint = {}

    def calculate(self, x, date):
        specicDate = convert_date_to_int(datetime.datetime.strptime(date, '%Y-%m-%d'))
        if x in self.exceptionPoint.keys():
            for cost, dates in self.exceptionPoint[x].items():
                if len(dates) == 0:
                    return cost

                if specicDate in dates:
                    return cost

        return round(self.A * x + self.b, self.precision)

    def validation(self, x, y, date):
        i = 0
        result = True
        while i < len(x):
            calculate_y = self.calculate(x[i][0], date[i])
            if calculate_y != y[i][0]:
                result = False
                # print('{}, {}, {}'.format(x[i][0], y[i][0], calculate_y))
            i += 1
        return result

    def __str__(self):
        return '{:.4f} * x {:+.4f}'.format(self.A, self.b)

In [29]:
def convert_date_to_int(dt):
    t = dt.year * 10000 + dt.month * 100 + dt.day
    return t
def calcuate_slope(adjust_X, adjust_Y, dic_slope, dic_slope_point, compareCount):
    counter = compareCount
    while counter < len(adjust_X):
        if adjust_X[counter] - adjust_X[counter - compareCount] != 0 and adjust_Y[counter] != 0 and adjust_Y[
            counter - compareCount] != 0 and adjust_X[counter] != 0 and adjust_X[counter - compareCount] != 0:
            slope = '{:.4f}'.format((adjust_Y[counter] - adjust_Y[counter - compareCount]) / (
                    adjust_X[counter] - adjust_X[counter - compareCount]))
            if not slope in dic_slope:
                dic_slope[slope] = 1
                dic_slope_point[slope] = []
            else:
                dic_slope[slope] += 1

            dic_slope_point[slope].append((counter, counter - compareCount))
        counter += 1
    return
def getMaxPrecision(dataList):
    maxPrecison = 0
    for data in dataList:
        stringValue = str(data)
        pointIndex = stringValue.find('.')
        pointLengh = 0
        if pointIndex != -1:
            pointLengh = len(stringValue[pointIndex + 1:])

        if maxPrecison < pointLengh:
            maxPrecison = pointLengh
    return maxPrecison
def calculateLinear(x, y, date):
    '''
    # logic -> y = Ax +B
    # 1. remove duplicate and sort the result by x
    # 2. calculate the confficients for each x(with x-1,x-2,x-3)
    # 3. sort the confficients to find the max value, marke it as A
    # 4. use the fitable result to find the B with biggest hit ratio
    # 5. time complexity: N
    :param x:
    :param y:
    :param date:
    :return:
    '''
    import operator
    # remove duplicate
    combineXY = np.concatenate((x, y), axis=1)
    combineXY = np.unique(combineXY, axis=0)
    # order by X
    combineXY = np.array(sorted(combineXY, key=lambda entry: entry[0]))
    # print(combineXY)
    # change it back to X, Y
    combineXY = np.hsplit(combineXY, 2)
    adjust_X = combineXY[0].flatten()
    adjust_Y = combineXY[1].flatten()
    # get max precision
    data_rand = getMaxPrecision(adjust_Y)
    dic_slope = {}
    dic_slope_point = {}
    calcuate_slope(adjust_X, adjust_Y, dic_slope, dic_slope_point, 1)
    calcuate_slope(adjust_X, adjust_Y, dic_slope, dic_slope_point, 2)
    calcuate_slope(adjust_X, adjust_Y, dic_slope, dic_slope_point, 3)
    final_a_key = max(dic_slope.items(), key=operator.itemgetter(1))[0]
    final_a = float(final_a_key)
    dic_b_fitCount = {}
    for (point2, point1) in dic_slope_point[final_a_key]:
        final_b = adjust_Y[point2] - final_a * adjust_X[point2]
        result = np.array([final_a * n + final_b for n in adjust_X]).round(data_rand) - adjust_Y
        dic_b_fitCount['{:.4f}'.format(final_b)] = len(result) - np.count_nonzero(result)
    max_b = float(max(dic_b_fitCount.items(), key=operator.itemgetter(1))[0])
    # print(dic_slope)
    # print(dic_b_fitCount)
    predic_y = np.array([final_a * n + max_b for n in adjust_X]).round(data_rand)
    fitRatio = 100 * (len(adjust_X) - np.count_nonzero(predic_y - adjust_Y)) / len(adjust_X)
    predicData_y = np.array([final_a * n + max_b for n in x]).round(data_rand)
    fitDataRatio = 100 * (len(x) - np.count_nonzero(predicData_y - y)) / len(x)

    # print(adjust_Y)
    # print(predic_y)
    # construct mapping funcion object
    exceptionPoint = {}
    i = 0
    liner_XPoints = set()
    while i < len(y):
        if abs(round(predicData_y[i][0], data_rand) - y[i][0]) != 0:
            if not x[i][0] in exceptionPoint.keys():
                exceptionPoint[x[i][0]] = {}

            if not y[i][0] in exceptionPoint[x[i][0]].keys():
                exceptionPoint[x[i][0]][y[i][0]] = set()

            exceptionPoint[x[i][0]][y[i][0]].add(
                convert_date_to_int(datetime.datetime.strptime(date[i], '%Y-%m-%d')))
        else:
            liner_XPoints.add(x[i][0])
        i += 1

    for x_cost, mappings in exceptionPoint.items():
        if x_cost not in liner_XPoints:
            if len(mappings) == 1:
                for y_cost, datas in mappings.items():
                    datas.clear()

    mappingFunction = MappingFunction()
    mappingFunction.A = final_a
    mappingFunction.b = max_b
    mappingFunction.precision = data_rand
    mappingFunction.exceptionPoint = exceptionPoint
    return mappingFunction, adjust_X, predic_y, fitRatio, fitDataRatio
def marge_rt_rp(group_rate_plan_ids):
        if search_level == "Room":
            read_data_rt, read_data_rp = read_rt_rp_by_room_id(search_id)
        if search_level == "Hotel":
            read_data_rt, read_data_rp = read_rt_rp_by_hotel_id(search_id)
        read_data_hilton = pd.merge(read_data_rt, read_data_rp, how='inner', left_on='RoomTypeID', right_on='RoomTypeID')
        read_data_hilton.rename(columns={'SKUGroupID': 'HotelId'}, inplace=True)
        logger.debug(read_data_hilton)
        input_data = read_data_hilton.loc[read_data_hilton['RatePlanID'].isin(group_rate_plan_ids)]
        return input_data
def calc_delta(root_no, child_no, abp_df):
    root = abp_df[abp_df['RatePlanID'] == root_no].reset_index(drop=True)
    child = abp_df[abp_df['RatePlanID'] == child_no].reset_index(drop=True)
    root = root.T
    child = child.T
    root.columns = ['Value']
    child.columns = ['Value']
    root['Value'] = root['Value'].apply(str)
    child['Value'] = child['Value'].apply(str)
    root = root.drop('RatePlanID')
    child = child.drop('RatePlanID')
    root['Value2'] = child['Value']
    root['ValueMatch'] = np.where(root['Value'] == child['Value'], True, False)
    root['ValueDiff'] = np.where(root['Value'] == child['Value'], '',
                                 root['Value'] + '→' + child['Value'])
    root = root.loc[root['ValueMatch'] == False]
    root.drop(['Value', 'Value2', 'ValueMatch'], axis=1, inplace=True)
    delta = re.sub(' +', ':', root['ValueDiff'].to_string())
    print("delta:{}".format(delta))
    return delta
def linear_prediction(rate_plan_list_ids, read_data):
    input_data = marge_rt_rp(rate_plan_list_ids)
    RP1 = rate_plan_list_ids[0]
    rp1_dd = read_data.loc[read_data['RatePlanID'] == RP1].set_index('StayDate')
    rp_func = pd.DataFrame()
    mappingFunctionResult = pd.DataFrame()
    data_length = len(rate_plan_list_ids)
    if data_length <= 1:
        return mappingFunctionResult
    count = 0
    row_size = math.ceil((data_length-1) ** 0.5)
    column_size = math.ceil((data_length-1) / row_size)
    fig, axes = plt.subplots(row_size, column_size, figsize=(column_size*4, row_size*5))
    plt.subplots_adjust(left=0.125, bottom=0.04, right=0.9, top=1, hspace=0.2, wspace=0.3)
    # 设置主标题
    fig.suptitle('x_rp:{}'.format(RP1))
    for i in range(row_size):
        logger.debug("left compare length: {}".format(data_length - count))
        for j in range(0, column_size):
            count += 1
            if count >= data_length:
                continue
            RP2 = rate_plan_list_ids[count]
            delta = calc_delta(RP1, RP2, input_data)
            rp2_dd = read_data.loc[read_data['RatePlanID'] == RP2].set_index('StayDate')
            rp_ds = pd.merge(rp1_dd, rp2_dd, on='StayDate')
            if rp_ds.empty:
                continue
            X = rp_ds[Observe + '_x'].to_numpy().reshape(-1, 1)
            y = rp_ds[Observe + '_y'].to_numpy().reshape(-1, 1)
            date = rp_ds.index.get_level_values('StayDate').values
            lr = LinearRegression().fit(X, y)
            pred_y = lr.predict(X)
            rp_func = rp_func.append(
                [[RP2, '{:.4f}'.format(lr.score(X, y)),
                  '{:.4f} * x {:+.4f}'.format(lr.coef_[0][0], lr.intercept_[0])]],
                ignore_index=True)

            (mappingFunction, adjust_X, predic_y, fitRatio, fitDataRatio) = calculateLinear(X, y, date)
            logger.debug("mappingFunction.exceptionPoint: {}".format(mappingFunction.exceptionPoint))
            dumpFunction = pickle.dumps(mappingFunction)
            mappingFunctionResult = mappingFunctionResult.append(
                [[RP1, RP2, len(pickle.dumps(X)), len(pickle.dumps(y)),
                  len(dumpFunction),
                  mappingFunction.validation(X, y, date)]],
                ignore_index=True)
            # 设置子标题
            # axes[i][j].set_title('{:.4f} * x {:+.4f}, fitPointRatio:{:.2f}%, fitDataRatio:{:.2f}%, y_rp:{}'.format(
            #     mappingFunction.A, mappingFunction.b, fitRatio, fitDataRatio, RP2))
            if column_size == 1 and row_size == 1:
                axes.set_title('y_rp:{}'.format(RP2))
                axes.scatter(X, y, color='blue', s=10)
                axes.plot(adjust_X, predic_y, color='red', linewidth=1)
                axes.text(X.min(), y.mean(), delta)
            if column_size == 1 and row_size > 1:
                axes[i].set_title('y_rp:{}'.format(RP2))
                axes[i].scatter(X, y, color='blue', s=10)
                axes[i].plot(adjust_X, predic_y, color='red', linewidth=1)
                axes[i].text(X.min(), y.mean(), delta)
            if column_size > 1 and row_size > 1:
                axes[i][j].set_title('y_rp:{}'.format(RP2))
                axes[i][j].scatter(X, y, color='blue', s=10)
                # axes[i][j].plot(X, pred_y, color='green', linewidth=1)
                axes[i][j].plot(adjust_X, predic_y, color='red', linewidth=1)
                axes[i][j].text(X.min(), y.mean(), delta)
            rp_ds.to_csv(
                '{}{}_Group{}_Line{}_{}_xy.csv'.format(PATTERN_MAPPING_OUTPUT_FOLDER, search_id, global_bast_group_id,
                                                       count,
                                                       Observe),
                index=False)
    # plt.tight_layout(w_pad=1.0)
    plt.savefig(OUTPUT_LINEAR_FILE_NAME.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, search_id), format='jpg',
                dpi=300)
    plt.show()
    plt.close()
    mappingFunctionResult.columns = ['BaseRP', 'ChildRP', 'BaseSize', 'ChildSize', 'MappingFunctionSize', 'Validation']
    mappingFunctionResult.to_csv(
        '{}{}_{}_{}_mappingFunction.csv'.format(PATTERN_MAPPING_OUTPUT_FOLDER, search_id, global_bast_group_id,
                                                Observe), index=False)
    return mappingFunctionResult

In [26]:
import pandas as pd
from reportlab.lib import colors
from reportlab.platypus import Table
from reportlab.pdfbase import pdfmetrics
from reportlab.lib.units import mm, inch
from reportlab.lib.enums import TA_CENTER
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.graphics.shapes import Drawing
from reportlab.lib.pagesizes import elevenSeventeen
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.graphics.charts.legends import Legend
from reportlab.graphics.charts.barcharts import VerticalBarChart
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image

# 注册字体
# 支持中文下载SimSun.ttf字体，并把它放在/ python3.7/site-packages/reportlab/fonts文件夹下
# https://github.com/StellarCN/scp_zh/tree/master/fonts
pdfmetrics.registerFont(TTFont('SimSun', 'SimSun.ttf'))


class Graphs:
    def __init__(self):
        pass

    # 绘制标题
    @staticmethod
    def draw_title():
        style = getSampleStyleSheet()
        ct = style['Title']
        ct.fontName = 'SimSun'
        ct.fontSize = 20
        # 设置行距
        ct.leading = 50
        # 颜色
        ct.textColor = colors.grey
        # 居中
        ct.alignment = 1
        # 添加标题并居中
        title = Paragraph('Mapping Function Research Report', ct)
        return title

    # 绘制内容
    @staticmethod
    def draw_text(content):
        style = getSampleStyleSheet()
        # 常规字体(非粗体或斜体)
        ct = style['BodyText']
        # 使用的字体s
        ct.fontName = 'SimSun'
        ct.fontSize = 14
        # 设置自动换行
        ct.wordWrap = 'CJK'
        # 居左对齐
        ct.alignment = 0
        # 第一行开头空格
        ct.firstLineIndent = 32
        # 设置行距
        ct.leading = 30
        text = Paragraph(content, ct)
        return text

    # 绘制表格
    @staticmethod
    def draw_table(*args):
        col_width = 120
        style = [
            ('FONTNAME', (0, 0), (-1, -1), 'SimSun'),  # 字体
            ('BACKGROUND', (0, 0), (-1, 0), '#d5dae6'),  # 设置第一行背景颜色
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # 对齐
            ('VALIGN', (-1, 0), (-2, 0), 'MIDDLE'),  # 对齐
            ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),  # 设置表格框线为grey色，线宽为0.5
        ]
        table = Table(args, colWidths=col_width, style=style)
        return table

    # 创建图表
    @staticmethod
    def draw_bar(bar_data=[], ax=[], items=[], max_value=100):
        drawing = Drawing(440, 200)
        drawing.hAlign = 'CENTRE'
        drawing.vAlign = 'TOP'
        bc = VerticalBarChart()
        bc.x = 35
        bc.y = 40
        # bc.y = y_axis
        bc.height = 200
        bc.width = 440
        bc.data = bar_data
        bc.strokeColor = colors.black
        bc.valueAxis.valueMin = 0
        bc.valueAxis.valueMax = max_value
        bc.valueAxis.valueStep = round(max_value / 10)
        bc.valueAxis.valueMax = max_value + bc.valueAxis.valueStep
        bc.categoryAxis.labels.dx = 8
        bc.categoryAxis.labels.dy = -10
        bc.categoryAxis.labels.angle = 20
        bc.categoryAxis.categoryNames = ax
        # 图示
        leg = Legend()
        leg.fontName = 'SimSun'
        leg.alignment = 'right'
        leg.boxAnchor = 'ne'
        leg.x = 465
        leg.y = 220
        leg.dxTextSpace = 10
        leg.columnMaximum = 3
        leg.colorNamePairs = items
        drawing.add(leg)
        drawing.add(bc)
        return drawing

In [27]:
linear_relationship_image = OUTPUT_LINEAR_FILE_NAME.format(PATTERN_ATTRIBUTE_OUTPUT_FOLDER, search_id)
mapping_function_file = '{}{}_{}_{}_mappingFunction.csv'.format(PATTERN_MAPPING_OUTPUT_FOLDER, search_id, global_bast_group_id,Observe)
if os.path.exists(linear_relationship_image) and os.path.exists(mapping_function_file) and enableCache=='true':
    logger.debug("Mapping function file already exist!!")
    mappingFunctionResult = pd.read_csv(mapping_function_file, encoding='utf-8', sep=',', engine='python', header=0).fillna(0)
    generate_report(mappingFunctionResult)
else:
    rate_plan_list_ids, read_data = read_and_preprocess_csv_file()
    mappingFunctionResult = linear_prediction(rate_plan_list_ids, read_data)
    generate_report(mappingFunctionResult)

delta:CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→166629
delta:RoomTypeID:166628→166629
CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→193407
delta:RoomTypeID:166628→193407
CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→193413
delta:RoomTypeID:166628→193413
CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→200160226
delta:RoomTypeID:166628→200160226
CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→200160278
delta:RoomTypeID:166628→200160278
CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→200160302
delta:RoomTypeID:166628→200160302
CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→200160344
delta:RoomTypeID:166628→200160344
CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→200160364
delta:RoomTypeID:166628→200160364
CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→200160421
delta:RoomTypeID:166628→200160421
CostCodeDefault:OD09X4→OD09XA
delta:RoomTypeID:166628→201119434
delta:RoomTypeID:166628→201119434
CostCodeDefault:OD09X4→OD09XA
de